In [ ]:
%matplotlib widget
from IPython.display import display, HTML
display(HTML("<style>.container{width:100% !important;}</style>"))

# Getting desired results: Tuning
This tutorial introduces some guidelines with regard to constraint weights and annealing temperatures to receive the desired annealing results. We demonstrate this running a traveling salesperson problem as an example.

### Preparation

We import the necessary libraries for Digital Annealer access. In the following form all classes are available without prefix in the current namespace.

## Traveling Salesman Problem as Example

The base usecase of this tutorial is the Traveling Salesman Problem (TSP). Given $N > 0$ cities, a salesman wants to find the shortest route to visit all cities exactly once within a round trip. In particular, the cities have varying distances towards each other. The salesman wants to start and end his trip in a particular city which you might think of as the depo.
For simplicity we drop the constraint that the salesman starts and ends his trip in a particular city, but just formulate the equivalent problem to visit the cities in a round trip no matter which city we start at.

This combinatorial optimization problem can be formulated as a quadratic unconstrained binary optimization problem (QUBO).
More precisely, all constraints and the optimization target are formulated with $N\cdot N$ binary variables $x_{t,c}\in \{0,1\}$, where $t\in \{0,\ldots, N-1\}$ stands for time and $c\in \{0,\ldots, N-1\}$ stands for city, and 
$$ 
x_{t,c} = \left\{ \begin{matrix}
                1, &&\text{if the salesman visits city }c\text{ at time }t,\\
                0 &&\text{else.}
          \end{matrix}\right.
$$

In the following, we initialize the `cities` and the `distance_table` containing all distances between cities. We also define a so called `VarShapeSet` which contains the shapes of this problems variables. You don't have to understand the code in detail. All qubo terms in this tutorial are predefined. Your task is to norm and weight the terms appropriately.

In [ ]:
import matplotlib.pyplot as plt

from dadk.Optimizer import *

from random import random, seed, randint

from tabulate import tabulate
import networkx as nx

class Optimizer_Demo_TSP_Model(OptimizerModel):
    ##################################################################################################################
    # constructor (defines GUI and test declaration)
    ##################################################################################################################
    def __init__(self, persistent_file="O_01_Optimizer_Demo_TSP.dao"):
        OptimizerModel.__init__(
            self,
            name = 'Demo_UIFunctions',
            load_parameter=[
                {'name':'shape', 'type':'select', 'value':'circle', 'options': ['circle', 'triangle', 'random'], 
                 'label':'Shape', 'description':'Select a shape to arrange the generated cities on a map'},
                {'name':'N', 'type':'int_slider', 'value':10, 'min':3, 'max': 90, 
                 'label':'Size', 'description':'problem size', 'tags':['demo1', 'demo2'],
                 'on_change': "fixed_city.max = fixed_time.max = new_value - 1"
                },
                {'name':'fixed_city', 'type':'int_slider', 'value':0, 'min':0, 'max':90, 
                 'label':'fixed city', 'description': '', 'tags':['demo1']},
                {'name':'fixed_time', 'type':'int_slider', 'value':0, 'min':0, 'max':90, 
                 'label':'fixed time', 'description':'', 'tags':['demo1']}
            ],            
            build_qubo_parameter=[
                {'name':'norm_time', 'type':'check', 'value':True, 
                 'label':'Norm $P_{time}$', 'description':'Check box to norm time polynomial to maximum absolute coefficient value of 1'},
                {'name':'norm_city', 'type':'check', 'value':True, 
                 'label':'Norm $P_{city}$', 'description':'Check box to norm city polynomial to maximum absolute coefficient value of 1'},
                {'name':'norm_distance', 'type':'check', 'value':True, 
                 'label':'Norm $P_{distance}$', 'description':'Check box to norm distance polynomial to maximum absolute coefficient value of 1'},
                {'name':'factor_A', 'type':'float_slider', 'value':2.1, 'min':0.0, 'max':10**2, 'step':0.01,
                 'label':'A (factor for $P_{time}$)', 'description':'Scaling factor A for time polynomial'},
                {'name':'factor_B', 'type':'float_slider', 'value':2.1, 'min':0.0, 'max':10**2, 'step':0.01,
                 'label':'B (factor for $P_{city}$)', 'description':'Scaling factor B for city polynomial'},
                {'name':'factor_C', 'type':'float_slider', 'value':1.0, 'min':0.0, 'max':10**2, 'step':0.01,
                 'label':'C (factor for $P_{distance}$)', 'description':'Scaling factor C for distance polynomial'},
                {'name':'factor_Z', 'type':'float_log_slider', 'value':1000.0, 'min':-3, 'max':20, 'base': 10.0, 'step':0.1,
                 'label':'Z (global factor)', 'description':'Scaling factor for combined polynomial', 'format': '.3e'},
                {'name':'show_polynomials', 'type':'check', 'value':False, 
                 'label':'Show polynomials', 'description':'Show polynomials. Not recommended for bigger scenarios'}
            ], 
            #
            # Define default values for the "Solve annealing" tab.
            # The user can modify these fields, or press the button 
            # "Default" and get these values again.
            #
            default_solver_parameter={
                'number_iterations':2000,
                'temperature_start':5000,
                'temperature_end':5,
                'offset_increase_rate':150.0},
            #
            # Define calculated values for the "Solve annealing" tab.
            # The user CAN'T modify these fields any more. The value
            # shown here is a default, which is will be overwritten with 
            # the methode self.Optimizer.set_calculated_solver_parameter(...)
            # see end of build_qubo for an example.
            #
            # calculated_solver_parameter={'temperature_start': 10000},
            demo_solver_parameter=[{'name':'number_iterations', 'tags':['demo1', 'demo2']},
                                   {'name':'number_runs', 'tags':['demo1']},
                                   'number_replicas'
                                  ],
            # input data for pareto test series
            pareto_setting={
                'var_object':['build_qubo_parameter', 'build_qubo_parameter', 'build_qubo_parameter'],
                'var_attribute':['factor_A', 'factor_B', 'factor_C'],
                'var_values':[
                    [50, 60, 80, 90, 100], [50, 60, 80, 90, 100],
                    [1.75, 1.50, 1.00, 0.75, 0.50]
                ]
            },
            # input data for performance test series
            timeseries_setting={
                'var_object':['load_parameter'],
                'var_attribute':['N'],
                'var_values':[[ 4, 8, 12, 16, 20, 24]],
                'max_bits': 8192        
            },
            persistent_file=persistent_file
        )

    
    ##################################################################################################################
    # load method (defines content of tab Setup scenario)
    ##################################################################################################################
    def load(self, shape='circle', N=10, fixed_city=0, fixed_time=0, silent=False):
        
        # store paramters in model attributes
        self.N = N
        self.fixed_city = min(fixed_city, N - 1)
        self.fixed_time = min(fixed_time, N - 1)
        
        # calculate distance table
        if shape == 'circle':
            self.cities = [[100*math.sin(c*2*math.pi/N), 100*math.cos(c*2*math.pi/N)] for c in range(N)]

        elif shape == 'triangle':
            self.cities = [[100 * c / (N - 2), 0] for c in range(N - 1)] + [[50, 100]]
            
        elif shape == 'random':
            seed(42)
            self.cities = [[100 * random(), 100 * random()] for c in range(N)] 
            
        self.distance_table = [[math.sqrt((self.cities[c_1][0] - self.cities[c_0][0])**2 + (self.cities[c_1][1] - self.cities[c_0][1])**2)
                                 for c_0 in range(N)] for c_1 in range(N)]
        
        # report actions to tab
        self.LOGGER.info(f'Model for circular map with {self.N} cities created.')
        
        if not silent:
            print(f"""
Model for map created:
    Number of cities: {self.N:3d}
    Shape:            {shape}
    Fixed city:       {self.fixed_city:3d}
    Fixed time:       {self.fixed_time:3d}
""")
        # rich output to linked file
        with open("./O_01_distance_table.html", 'w') as detail:
            detail.write('<h1>Distances:</h1>' + tabulate(self.distance_table, range(N), showindex="always", tablefmt="html"))
        self.Optimizer.set_load_details_reference(url = "O_01_distance_table.html")
        
    ##################################################################################################################
    # build_qubo method (defines content of tab Build QUBO)
    ##################################################################################################################
    def build_qubo(self, norm_time=False, norm_city=False, norm_distance=False, factor_A=1.0, factor_B=1.0, factor_C=1.0, factor_Z=1.0, show_polynomials=False, silent=False):
        
        # variable bit as default are initialized with -1
        constant_bits = np.full((self.N, self.N), -1, np.int8)
        # all bits for fixed_time are set to constant 1
        constant_bits[self.fixed_time, :] = 0
        # all bits for fixed city are set to constant 0
        constant_bits[:, self.fixed_city] = 0
        # bit for fixed_time in fixed_city is set to constant 1
        constant_bits[self.fixed_time, self.fixed_city] = +1                
        # square array x with constant bits is created as VarShapeSet            
        self.qubo_vs = VarShapeSet(BitArrayShape('x', (self.N, self.N), constant_bits=constant_bits, axis_names=['Time', 'City']))        
                        
        #-P_time-------------------------------------------------------------------------------
        
        self.P_time = BinPol(self.qubo_vs)   
        for t in range(self.N):
            self.P_time += (1 - BinPol.sum(Term(1, (('x', t, c),), self.qubo_vs) for c in range(self.N))) ** 2
        if norm_time:
            self.P_time *= 0.5
        self.P_time *= factor_A
        
        #-P_city-------------------------------------------------------------------------------

        self.P_city = BinPol.sum(
            (1 - BinPol.sum(Term(1, (('x', t, c),), self.qubo_vs) for t in range(self.N))) ** 2 
            for  c in range(self.N))
        if norm_city:
            self.P_city.normalize()
        self.P_city *= factor_B

        #-P_distance---------------------------------------------------------------------------
                        
        self.P_distance = BinPol.sum(Term(self.distance_table[c_0][c_1], (('x', t, c_0), ('x', (t + 1) % self.N, c_1)), self.qubo_vs)
                                                       for c_0 in range(self.N) for c_1 in range(self.N) if c_0 != c_1 for t in range(self.N))
        if norm_distance:
            self.P_distance.normalize()
        self.P_distance *= factor_C

        #-HQ-----------------------------------------------------------------------------------
        
        self.HQ = factor_Z * (self.P_time + self.P_city + self.P_distance)

        self.recommended_scale_factor_16 = self.HQ.get_scale_factor(bit_precision=16) * factor_Z
        self.recommended_scale_factor_32 = self.HQ.get_scale_factor(bit_precision=32) * factor_Z
        self.recommended_scale_factor_64 = self.HQ.get_scale_factor(bit_precision=64) * factor_Z
        
        
        if not silent:
            print(f"""
    Polynomial created 
    Number of bits:               {self.HQ.N: 5d}
    Scale factor Z:               {factor_Z: .3e}
    Recommended scale factor 16:  {self.recommended_scale_factor_16: .3e}
    Recommended scale factor 32:  {self.recommended_scale_factor_32: .3e}
    Recommended scale factor 64:  {self.recommended_scale_factor_64: .3e}
    """)
            if show_polynomials:
                poly_tabs = OutputTab('P_time', 'P_city', 'P_distance', 'HQ')
                with poly_tabs.tab_out['P_time']:
                    print('A * P_time: ', self.P_time, '\n')
                with poly_tabs.tab_out['P_city']:
                    print('B * P_city: ', self.P_city, '\n')
                with poly_tabs.tab_out['P_distance']:
                    print('C * P_distance: ', self.P_distance, '\n')
                with poly_tabs.tab_out['HQ']:
                    print('Z * HQ: ', self.HQ, '\n')
                poly_tabs.display()
    
    ##################################################################################################################
    # prep_result method (evaluates result of annealing defines content of tab Solve annealing)
    ##################################################################################################################
    def prep_result(self, solution_list:SolutionList, silent=False):

        configuration = solution_list.min_solution.configuration

        self.configuration = configuration # <-- best solution
        self.start_table = None
        self.route = None
              
        if not silent:
            e_HQ = self.HQ.compute(self.configuration)
            e_time = self.P_time.compute(self.configuration)
            e_city = self.P_city.compute(self.configuration)
            e_distance = self.P_distance.compute(self.configuration)
            self.start_table = solution_list.min_solution.extract_bit_array('x')
            
            print('Energy of QUBO / parts in best solution (P_time and P_city should be zero)')
            print("HQ         = %10.2F" % e_HQ)
            print("P_time     = %10.2F" % e_time)
            print("P_city     = %10.2F" % e_city)
            print("P_distance = %10.2F" % e_distance)
            print("")
            
            if e_time < 0.01 and e_city < 0.01:

                self.route = [np.argmax(self.start_table.data[t,:]) for t in range(self.N)]

                distances = [self.distance_table[self.route[t]][self.route[(t + 1) % self.N]] for t in range(self.N)]
                self.output_table = tabulate(zip(range(self.N),
                                                 self.route,
                                                 self.route[1:] + self.route[:1],
                                                 distances,
                                                 np.cumsum(distances)
                                                ),
                                             ['t', 'from', 'to', 'distance', 'total'], tablefmt="html")
                display(HTML(self.output_table))
            else:
                print("invalid solution, try again!")
           
    ##################################################################################################################
    # report method (defines content of Report tab)
    ##################################################################################################################
    def report(self, silent=False):
        
        if self.start_table is not None:
            self.start_table.draw(figsize=(6.0, 6.0))
            
    def report_title(self):
        return "Bit map report"
    
    ##################################################################################################################
    # report1 method (defines content of additional Report tab)
    ##################################################################################################################
    def report1(self, silent=False):
        print('This is the best found roundtrip:')
        if self.start_table is not None:
            display(HTML(self.output_table))
            
    def report1_title(self):
        return "Route report"
    
    ##################################################################################################################
    # draw method (defines content of visualization tab)
    ##################################################################################################################
    def draw(self):
        if self.start_table is not None and self.route is not None:
            self.graph = nx.Graph()

            nodes={}
            for i in range(self.N):
                nodes[i] = str(i);
                self.graph.add_node(nodes[i], pos=(self.cities[i][0], self.cities[i][1]))

            path = [[self.route[t], self.route[(t + 1) % self.N]] for t in range(self.N)]

            for i in range(self.N):
                for j in range(i+1, self.N):
                    if [i, j] in path or [j, i] in path: 
                        self.graph.add_edge(nodes[min(i,j)], nodes[max(i,j)], weight=10)
                    else:
                        self.graph.add_edge(nodes[min(i,j)], nodes[max(i,j)], weight=1)

            fig = plt.figure(num='Graph (created at %s)' % datetime.now(), figsize=(8.0, 8.0))

            pos=nx.get_node_attributes(self.graph,'pos')

            # nodes
            nx.draw_networkx_nodes(self.graph, pos, nodelist=self.graph.nodes, node_size=100, node_color="red")

            # edges        
            edges_bold = [(u, v) for (u, v, d) in self.graph.edges(data=True) if d['weight'] == 10]
            nx.draw_networkx_edges(self.graph, pos, edgelist=edges_bold, width=3, edge_color="black")

            edges_thin = [(u, v) for (u, v, d) in self.graph.edges(data=True) if d['weight'] == 1]
            nx.draw_networkx_edges(self.graph, pos, edgelist=edges_thin, width=1, alpha=0.1, edge_color="black")

            # labels
            nx.draw_networkx_labels(self.graph, pos, font_size=6, font_family='sans-serif')

            plt.axis('off')
            plt.show()
        
    ##################################################################################################################
    def pareto(self):
    ##################################################################################################################

        return {
            'HQ' : self.HQ.compute(self.configuration),
            'P_time': self.P_time.compute(self.configuration),
            'P_city': self.P_city.compute(self.configuration),
            'P_distance': self.P_distance.compute(self.configuration)
        }

## Norming 

In order to simplify the problem of finding appropriate weighting factors for the polynomials, we first want to norm the different terms such that their maximal coefficients are equal to one. 

The first constraint is that at each time $t$ exactly one city $c$ must be visited. This condition can be forced by the penalty term 
$$
P_{time} = \sum_{t=0}^{N - 1} \left(1 - \sum_{c=0}^{N - 1} x_{t,c} \right)^2. \label{eq1}\tag{1}
$$
You can norm the polynomial `P_time` using its `multiply_scalar` method, so that its maximal coefficient equals one. This is easy, since you know, that the maximum coefficient is 2.

The second constraint is that each city $c$ must be visited exactly once within the time steps $t\in \{0,\ldots,N-1\}$. This condition can be forced by the penalty term 
$$
P_{city} = \sum_{c=0}^{N - 1} \left(1 - \sum_{t=0}^{N - 1} x_{t,c} \right)^2. \label{eq2}\tag{2}
$$
For `P_city` the biggest coefficient is 2 as well. We use `multiply_scalar` method with 0.5, so that its maximal coefficient equals one.

Finally, the optimization target for this problem is to find a roundtrip which minimizes the overall travelled distance, i.e. to minimize
$$
P_{distance} = \sum_{t=0}^{N - 1} 
                \sum_{c_0=0}^{N - 1} 
                 \sum_{c_1=0 \\c_0 \ne c_1}^{N - 1} 
                  \Delta_{c_0, c_1} x_{t,c_0} x_{(t+1) \% N, c_1}. \label{eq3}\tag{3}
$$
Please norm the polynomial `P_distance` using its `normalize` method, so that its maximal coefficient equals one.

The normalization can be done using the respective buttons in the ``Build QUBO`` tab.

In [ ]:
optimizer = Optimizer(Optimizer_Demo_TSP_Model())

## Weighting
With norming weights $w_{tnorm},w_{cnorm}$ and $w_{dnorm}$, and weighting factors $A,B,C>0$, the final polynomial to minimize will be
$$
\begin{align}
 HQ &= A \cdot w_{tnorm} P_{time}+ B w_{cnorm} P_{city}+ C w_{dnorm} \cdot P_{distance} \\
    &= A \bar{P}_{time}+ B \bar{P}_{city}+ C \bar{P}_{distance}. \label{eq4}\tag{4}
\end{align}   
$$
We have to find appropriate real numbers $A,B,C$ to enforce the penalty terms $P_{time}, P_{city}$ to become zero, while minimizing the optimization target $P_{distance}$. Without loss of generality we chose $C=1$ and $A=B$, since the structure of the penalty polynomials is identical. With $\bar{P}_{penalty}:= \bar{P}_{time}+ \bar{P}_{city}$ we get

$$
HQ = A \bar{P}_{penalty} + \bar{P}_{distance}. \label{eq5}\tag{5}
$$

The factor $A$ must be big enough to guarantee, that in every local minimum of $HQ$ the value of the penalty polynomial is zero, i.e. the solution always follows the constraints. Simultaneously we are looking for a preferably small $A>0$ to preserve as much influence of the optimization term as possible.

Assume an optimum round trip. If we drop the location with the longest traveling time to its predecessor and successor, then we miss one time step and one city. The penalty polynomials $P_{time}$ and $P_{city}$ return a value of $1$, so does the normed penalty polynomial $\bar P_{penalty}$ as their sum divided by 2. The win in traveling distance is less or equal two times the maximum distance between any two cities. For the normed distance polynomial this is a maximum win of $2$. The energy increase is at least $A-2$, which is positive for $A>2$ and therefore this invalid solution is not a global minimum. $A=2$ is not big enough: For an equilateral triangle with cities $0, 1, ...N-2$ on the base and $N-1$ in the top it is easy to verify, that for $A=2$ the energy of an optimum round trip state $X^o$ with $1=x^o_{0,0}=x^o_{1,1}= ... =x^o_{N-1,N-1}$ and all other bits zero and the energy of the invalid state $X^i$ with $1=x^i_{0,0}=x^i_{1,1}= ... =x^i_{N-2,N-2}$ are identical.

You can examine the influence of these weighting factors by setting the values for  $A$ ,  $B$  and  $C$  under the tab Build QUBO and then solving the QUBO. For the triangle with $A=B=2$ you should get invalid solutions for a global minimum from time to time. For reliable results select all norming options as true.

Above considerations indicated the need to select $A>2$ in order to strengthen the penalty part and avoid invalid solutions as global minimum. Vice versa it can be shown that $A>2$ is sufficient to avoid invalid solutions in strict local minima, so the annealing process will not stagnate in an invalid solution at a cold temperature in the end but will continue searching for better valid solutions. If you are not interested in the mathematical proof of this, you can continue reading at the beginning of the next chapter.

<u>Proposition</u>: Let $HQ=A \bar{P}_{penalty} + \bar{P}_{distance}$ be the QUBO for the TSP problem with normed penalty polynomial  $\bar{P}_{penalty}$ and normed optimization target $\bar{P}_{distance}$ according to $(\ref{eq5})$. Set the weight $A>2$ and let $\check{X}$ be a strict local minimum of $HQ$, then $\bar{P}_{penalty}({\check X}) = 0$, i.e. the constraints for a valid round trip are fulfilled.

<u>Proof</u>: Let us assume  $\bar{P}_{penalty}({\check X}) > 0$. Two cases are distinguished:  case (i) for $P_{time}({\check X}) > 0$ and case (ii) for $P_{time}({\check X}) = 0$.

For case (i) with $P_{time}(\check{X}) = \sum_{t=0}^{N-1} \left( 1 - \sum_{c=0}^{N-1} {\check x}_{t,c} \right)^2 >0$ again two sub cases are considered. In case (i.a) we assume there exits a time $\bar t$ with $\sum_{c=0}^{N-1} {\check x}_{{\bar t},c} > 1$. Let $\bar c$ be one of the cities with $ {\check x}_{{\bar t},{\bar c}} = 1$. For $X$ defined as

$$
\begin{equation}
    x_{t,c}=
    \begin{cases}
      {\check x}_{t,c}& \text{for } (t,c) \ne ({\bar t},{\bar c}) \\
      0& \text{for } (t,c) = ({\bar t},{\bar c}) 
    \end{cases}
\end{equation}
$$

it will be shown, that $HQ(X) \le HQ({\check X})$, and therefore ${\check X}$ would not be a strict local minimum. For $X$ we get

$$
\sum_{c=0}^{N-1} x_{{\bar t},c} =  \sum_{c=0}^{N-1} {\check x}_{{\bar t},c} - 1  > 0\\
\begin{align}
    \Rightarrow \left( \sum_{c=0}^{N-1}  x_{{\bar t},c} - 1 \right)^2 
                    &= \left( \sum_{c=0}^{N-1} {\check x}_{{\bar t},c} - 2 \right)^2\\
                    &\le \left( \sum_{c=0}^{N-1} {\check x}_{{\bar t},c} - 1 \right)^2 - 1\\
    \Rightarrow P_{time}(X) &\le P_{time}({\check X}) - 1 \label{eq6}\tag{6}
\end{align}
$$

Now when evaluating $P_{city}$ for $X$ and $\check X$ all sums over $t$ are identical except for $c=\bar c$. For $\sum_{t = 0}^{N-1}\check x_{t,\bar c} > 1$ again we can estimate

$$
\begin{align}
    \left( \sum_{t=0}^{N-1}  x_{t, {\bar c}} - 1 \right)^2 &\le \left( \sum_{t=0}^{N-1} {\check x}_{t,{\bar c}} - 1 \right)^2 - 1 \\
    \Rightarrow P_{city}(X) &\le P_{city}({\check X}) - 1 \label{eq7}\tag{7}
\end{align}
$$

In the case of $\sum_{t = 0}^{N-1}\check x_{t,\bar c} = 1$ we get $\sum_{t = 0}^{N-1} x_{t,\bar c} = 0$ and for the sums $\left( \sum_{t=0}^{N-1}\check  x_{t, {\bar c}} - 1 \right)^2 = 0$ and $\left( \sum_{t=0}^{N-1}  x_{t, {\bar c}} - 1 \right)^2 = 1$. This delivers 

$$
P_{city}(X) = P_{city}({\check X}) + 1 \label{eq8}\tag{8}
$$

and we get from $(\ref {eq7})$ and $(\ref {eq8})$ the inequality 

$$ 
P_{city}(X) \le P_{city}({\check X}) + 1  \label{eq9}\tag{9}
$$

and from $(\ref {eq6})$ and $(\ref {eq9})$ the inequality 

$$ 
P_{time}(X) + P_{city}(X) \le P_{time}({\check X}) + P_{city}({\check X}) \label{eq10}\tag{10}
$$

Since all coefficients of $P_{distance}$ are non negative, switching off a bit cannot enlarge the value of the polynomial, hence

$$ 
P_{distance}(X) \le P_{distance}({\check X})  \label{eq11}\tag{11}
$$

Multiplying $(\ref {eq10})$ by $\frac A 2$ and $(\ref {eq11})$ by $w_{dnorm}$ the addition of both delivers

$$
\begin{align}
    A \bar{P}_{penalty}(X) + \bar{P}_{distance}(X) &\le A \bar{P}_{penalty}({\check X}) + \bar{P}_{distance}({\check X})\\
    \Rightarrow HQ(X) &\le HQ({\check X}) \label{eq12}\tag{12}
\end{align}
$$

which contradicts our assumption, that $\check X$ is a strict local minimum of $HQ$.

For the sub case (i.b) for all times $t$ we have 

$$
\sum_{c=0}^{N-1}\check  x_{t, c} \le 1 \label{eq13}\tag{13}.
$$

Since $P_{time}(\check X) > 0$ there exists at least one $\bar t$ with $\sum_{c=0}^{N-1}\check  x_{\bar t, c} = 0$ and therefore the total number of bits switched on is $\sum_{t=0}^{N-1}\sum_{c=0}^{N-1}\check  x_{t, c} < N$. Hence there must be a city $\bar c$ with $\sum_{t=0}^{N-1}\check  x_{t, \bar c} = 0$. For $X$ defined by  

$$
\begin{equation}
    x_{t,c}=
    \begin{cases}
      {\check x}_{t,c}& \text{for } (t,c) \ne ({\bar t},{\bar c}) \\
      1& \text{for } (t,c) = ({\bar t},{\bar c}) 
    \end{cases}
\end{equation}
$$

it will be shown, that $HQ(X) < HQ({\check X})$, and therefore ${\check X}$ would not be a strict local minimum. The modification of $\check X$ to $X$ reduces in $P_{time}$ the square term $\left( 1 - \sum_{c=0}^{N-1}\check  x_{\bar t, c} \right)^2$ from 1 to 0 and the same for the square term $\left( 1 - \sum_{t=0}^{N-1}\check  x_{t, \bar c} \right )^2$ in $P_{city}$. Hence $P_{time}(X) = P_{time}(\check X) -1$ and $P_{city}(X) = P_{city}(\check X) -1$ and therefore

$$
A {\bar P}_{penalty}(X) = A {\bar P}_{penalty}({\check X}) -A. \label{eq14}\tag{14}
$$

Let $t_b$ and $t_a$ be the time indices before and after $\bar t$, i.e. $t_b + 1 \equiv \bar t (N)$ and $\bar t + 1 \equiv t_a (N)$. According to $(\ref{eq13})$ there is at most one $c_b$ and one $c_a$ with $x_{t_b, c_b} = 1$ and  $x_{t_a, c_a} = 1$. Using this we can calculate an upper estimation $\bar P_{distance}$:

$$
\begin{align}
    \bar P_{distance}(X) &= \omega_{dnorm}\sum_{t=0}^{N-1}\sum_{c_0=0}^{N-1}\sum_{c_1=0 \\c_1 \ne c_0}^{N-1} \Delta_{c_0, c_1} x_{t, c_0} x_{t+1 mod N, c_1} \\
                         &= \omega_{dnorm}\left( \sum_{t=0}^{N-1}\sum_{c_0=0}^{N-1}\sum_{c_1=0 \\c_1 \ne c_0}^{N-1} \Delta_{c_0, c_1} {\check x}_{t, c_0} {\check x}_{t+1 mod N, c_1}
                           + \Delta_{\bar c, c_a} x_{\bar t, \bar c} x_{t_a, c_a}
                           + \Delta_{c_b, \bar c} x_{t_b, c_b} x_{\bar t, \bar c}\right)\\
                         &\le {\bar P}_{distance}(\check X) + 2. \label{eq15}\tag{15}
\end{align}
$$

From $(\ref{eq14})$ and $(\ref{eq15})$ we get

$$
\begin{align}
    HQ(X) &= A {\bar P}_{penalty}(X) + \bar P_{distance}(X)\\
          &\le A {\bar P}_{penalty}({\check X}) - A + {\bar P}_{distance}(\check X) + 2\\
          &< HQ(\check X)
\end{align}
$$

in contradiction to the assumption, that $\check X$ is a strict local minimum of $HQ$.

For the case (ii) we have $P_{time}(\check X) = 0$ and therefore $P_{city}(\check X) > 0$. The subcase (ii.a) supposing a $\bar c $ with $\sum_{t=0}^{N-1}\check x_{t,\bar c} > 1$ can be proven analogously to (i.a) switching the roles of $P_{time}$ and $P_{city}$. $P_{distance}$ is not symmetric in $t$ and $c$ but we did only make use of the fact, that the coefficients are not negative; this can be argued in the same way for the proof of (ii.a). 

Lastly for the case (ii.b) we have $\sum_{t=0}^{N-1}\check x_{t,c}<1$ for all cities $c$. Since $P_{city}(\check X)>0$ there exists $\bar c$ with $\sum_{t=0}^{N-1}\check x_{t,\bar c}=0$. Therefore the number of bits switched on is $\sum_{c=0}^{N-1}\sum_{t=0}^{N-1}\check x_{t,c}<N$. This is not possible in case (ii) with $P_{time}(\check X)=0$ and therefore the case (ii.b) cannot occur. This completes our proof of the proposition. $\square$

In [ ]:
optimizer = Optimizer(Optimizer_Demo_TSP_Model())

## Global weighting factor

In the previous paragraph we discussed the weighting of penalty and optimization polynomial against each other to avoid invalid solutions without suppressing the optimization target excessively. Nevertheless in your experiments you might have faced invalid solutions though your norming and weight balancing was correctly defined. The reason is that the Digital Annealer rounds all coefficients to integers.

In order to compensate these rounding effects we have to find a global weighting factor $Z$ to be multiplied with the QUBO.
$$
HQ= Z( A w_{tnorm} P_{time}+ B w_{cnorm} P_{city}+ C w_{dnorm} P_{distance}).
$$

One effective measure is to make the minimal non zero coefficient in the final QUBO to have (at least) an absolute value of one. This can be reached by a factor $Z$ defined as
$$
Z := \left \lceil \frac 1 {\min \left\{|c_{i,j}| \,\middle\vert\, c_{i,j,} \ne 0 \text{ for } i,j=0, 1, ...N-1 \right\}} \right \rceil.
$$

A second approach to deal with the integer rounding is to multiply the final QUBO by a constant such that the minimal non-zero difference between two coefficients equals one. This reduces the computational bias of the Digital Annealer algorithm and delivers  better fidelity due to the given business problem. In particular for the TSP problem, different feasible configurations for the problem, which represent round trips with different overall traveled distance, should also have a different QUBO value. Weighting $HQ$ by the reciprocal of the minimal non-zero distance between two of its coefficients supports this:
$$
Z := \left \lceil \frac 1 {\min \left \{|c_{i,j}-c_{k,l}| \,\middle\vert\, c_{i,j} \ne c_{k,l} \text{ for } i,j,k,l=0, 1, ...N-1 \right \}} \right \rceil.
$$

When scaling up the QUBO it is important to consider the limits for the coefficients itself. It can be restricted to $16$, $32$ or $64$ bits. It is a good idea to scale as high as possible, hence the scaling factor can be derived from the bit precision $\nu = 16, 32, 64$:
$$
Z := \left \lfloor \frac {2^{\nu - 1} -1} {\max \left\{|c_{i,j}| \,\middle\vert\, c_{i,j,} \text{ for } i,j=0, 1, ...N-1 \right\}} \right \rfloor.
$$

The last approach is supported by the dadk. The class ``BinPol`` offers two methods ``get_scale_factor(bit_precision=16)`` and ``scale(bit_precision=16)`` to determine the scale factor and to directly apply the scale factor to the calling ``BinPol`` object. The demo example uses ``get_scale_factor`` to recommend possible scaling factors.

In [ ]:
optimizer = Optimizer(Optimizer_Demo_TSP_Model())

## Annealing temperatures

Now that the final QUBO is generated, appropriate annealing temperatures have to be defined. On the one hand, the start temperature of the annealing process must be large enough to search for the solution within the whole search space. On the other hand, the number of iterations are limited, so the start temperature should not be too large. Finally, the end temperature must be small enough for convergence of the annealing algorithm, but again not too small so that the algorithm is stuck in the final configuration for many iterations. The third parameter to take care of is the dynamic energy offset, which helps the algorithm at lower temperatures to escape from local minima. Is it too small, then the escape still will consume a lot of cycles; is it too large, then the escape step might randomly flip every bit, which distracts the random walk from better solutions nearby.

Use the following box to test some cases and figure out, how it correlates with the global weight $Z$.

In [ ]:
optimizer = Optimizer(Optimizer_Demo_TSP_Model())

### Search phases

A well adjusted search process typically has two phases. In the first phase the temperature is sufficiently high to escape instantly from local minima. In this phase the random walk has progress in every step. Note that the Digital Annealer with the parallel flipping attempts for all bits provides higher flip probability for lower temperatures compared to classical simulated annealing. Nevertheless there should be a second phase with temperatures low enough to produce waiting cycles before escaping from a local minimum. Both phases and the transition point from the first to the second phase can be controlled by the start and end temperature.

To estimate the dependency of the flip probability at a local minimum from the temperature let $X$ be a minimum state and $X^i$ be the state, for which the bit with index $i$ is flipped:
$$
\begin{equation}
    x^i_j=
    \begin{cases}
      x_j & \text{for } i \ne j \\
      1 - x_j & \text{for } i = j .
    \end{cases}
\end{equation}
$$

Let $T$ be the temperature and $E(X)$ and $E(X^i)$ the energies of the states. Since $X$ is a local minimum we can calculate the flip probability as follows:
$$
p_{flip} = 1 - \prod_{i=0}^{N-1} \left(1 - e^{- \frac {E(X_i)-E(X)} T} \right). \label{eq16}\tag{16}
$$

Let $\Delta E(X)$ be a random variable defined on the set of minima of E as the mean of energy differences for all bit flips:
$$
\Delta E(X) = \frac 1 N \sum_{i=0}^{N-1} (E(X_i)-E(X))
$$

As rough estimation we next replace all energy differences in $(\ref{eq16})$ by the expectation value of $\Delta E(X)$:
$$
\begin{align}
p_{flip} \approx 1 - \prod_{i=0}^{N-1} \left(1 - e^{- \frac {\mathbb{E}(\Delta E(X))} T} \right). \label{eq17}\tag{17}
\end{align}
$$

Resolving $(\ref{eq17})$ for $T$ delivers and estimation for a suitable temperature:
$$
T \approx - \frac {\mathbb{E}(\Delta E(X))} {\ln(1 - \sqrt[N]{1-p_{flip}})} . \label{eq18}\tag{18}
$$

### Start Temperature
In the beginning waiting cycles in local minima should be avoided. Therefore we define a high flip probability e.g. $p_{start}=0.99$. According to $(\ref{eq18})$ we get for the start temperature:
$$
T_{start} = - \frac {\mathbb{E}(\Delta E(X))} {\ln(1 - \sqrt[N]{1-p_{start}})} . \label{eq19}\tag{19}
$$

### End Temperature
For the transition point between the first and the second phase waiting cycles in local minima should become more likely. Therefore we define a lower flip probability e.g. $p_{trans}=0.5$. According to $(\ref{eq18})$ we get for the transition temperature:
$$
T_{trans} = - \frac {\mathbb{E}(\Delta E(X))} {\ln(1 - \sqrt[N]{1-p_{trans}})} . \label{eq20}\tag{20}
$$

Let $I$ be the total number of iterations. Let the transition point be defined by a part of iterations $0 < \nu \le 1$. Let $d:=1- decay$ be the factor multiplied for exponential temperature model in every step to the previous temperature. Then we have $T_{start}d^{I} = T_{end}$ and $T_{start}d^{\nu I} = T_{trans}$. The first equations gives $d^I = T_{start}^{-1} T_{end}$ and using this to replace $d^I$ in the second equation, we get $T_{start}  T_{start}^{-\nu} T_{end}^{\nu} = T_{trans}$. Therefore $T_{end} = T_{start}^{1- \frac 1 \nu} T_{trans}^{- \frac 1 \nu}$ and with $(\ref{eq20})$ we get:
$$
T_{end} = T_{start}^{1- \frac 1 \nu} \left( - \frac {\mathbb{E}(\Delta E(X))} {\ln(1 - \sqrt[N]{1-p_{trans}})} \right)^{- \frac 1 \nu}.  \label{eq21}\tag{21}
$$

### Offset Parameter

Finally we have to determine the dynamic energy offset, that should help to escape faster from local minima. To climb safely upwards from a local minimum, we add only a fraction of the expected depth of the minimum in every step. If we want to reach the rim in e.g. $k:=10$ steps, then we define 
$$
E_{dyn\_off} :=  \frac {\mathbb{E}(\Delta E(X))} k.
$$

Now you are ready to  determine good temperatures and energy offset for the annealing process. The only open point is the determination of  the expectation value of the mean bit flip energy delta in local minima. The ``Optimizer`` does this for you by sampling. In the ``Solve annealing`` tab you can set the intended probabilities, the transition point and properties to setup the sampling. Use the following box to get familiar with this way of temperature tuning. 

In [ ]:
optimizer = Optimizer(Optimizer_Demo_TSP_Model())